# Step 07 - Telemetry Simulator

Notebook below retrieves holdout data from delta lake and submits requests (one every 15 seconds) to endpoint which is subsequently captured by the endpoint

In [ ]:
from azure.ai.ml import MLClient, Input
from azure.ai.ml.entities import Data, ManagedOnlineEndpoint, ManagedOnlineDeployment, Model, Environment, ModelPackage, BaseEnvironment, AzureMLOnlineInferencingServer, CodeConfiguration, DataCollector, DeploymentCollection
from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import ContainerRegistryCredential

import requests
import json
import time

subscription_id = "..."
resource_group = "..."
workspace = "..."
model_name = "zone1-power-consumption-xgboost-model"

ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)
ml_client

In [2]:
import pandas as pd
from mltable import from_delta_lake
from datetime import datetime, timezone
mltable_ts = from_delta_lake(delta_table_uri="azureml://subscriptions/<SUB>/resourcegroups/<RG>/workspaces/<WORKSPACE>/datastores/<DATASTORE>/paths/holdout/", timestamp_as_of=datetime.now(timezone.utc).isoformat())
mltable_ts

df = mltable_ts.to_pandas_dataframe()

df

,DateTime,Temperature,Humidity,Wind-Speed,general-diffuse-flows,diffuse-flows,Zone-1-Power-Consumption,Zone-2--Power-Consumption,Zone-3--Power-Consumption
0,2017-10-01 00:00:00,18.120,87.6,4.916,0.080,0.163,31755.62363,17985.06224,14898.96657
1,2017-10-01 00:10:00,18.000,88.5,4.916,0.055,0.137,31175.84245,17611.61826,14531.30699
2,2017-10-01 00:20:00,18.010,88.4,4.916,0.069,0.148,30154.92341,17148.54772,14321.21581
3,2017-10-01 00:30:00,18.160,88.4,4.914,0.062,0.167,29764.20131,16827.38589,14198.66261
4,2017-10-01 00:40:00,18.180,89.1,4.917,0.051,0.115,29600.35011,16621.99170,13889.36170
...,...,...,...,...,...,...,...,...,...
13099,2017-12-30 23:10:00,7.010,72.4,0.080,0.040,0.096,31160.45627,26857.31820,14780.31212
13100,2017-12-30 23:20:00,6.947,72.6,0.082,0.051,0.093,30430.41825,26124.57809,14428.81152
13101,2017-12-30 23:30:00,6.900,72.8,0.086,0.084,0.074,29590.87452,25277.69254,13806.48259
13102,2017-12-30 23:40:00,6.758,73.0,0.080,0.066,0.089,28958.17490,24692.23688,13512.60504


In [ ]:
scoring_uri = ml_client.online_endpoints.get('power-endpoint').scoring_uri
api_key = ml_client.online_endpoints.get_keys('power-endpoint').primary_key

headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

while True:
    for idx, row in df.iterrows():
        row_data = dict(row.copy())
        del row_data['DateTime']
        response = requests.post(scoring_uri, data=json.dumps({'data': [dict(row_data)]}), headers=headers)
        if idx%5==0:
            print(idx)
            print(response.json())
        time.sleep(15)

0
[25861.083984375]
5
[30288.671875]
10
[26853.451171875]
15
[26034.640625]
20
[25602.8671875]
25
[26068.8828125]
30
[29367.853515625]
35
[26892.322265625]
40
[26713.54296875]
45
[27082.5078125]
50
[27116.234375]
55
[27638.12890625]
60
[29586.57421875]
65
[34167.0]
70
[34782.57421875]
75
[34980.44140625]
80
[35137.0625]
85
[33972.7265625]
90
[34438.39453125]
95
[34207.27734375]
100
[34345.12890625]
105
[34751.2890625]
110
[40480.8046875]
115
[37992.28515625]
120
[37609.19921875]
125
[39498.76953125]
130
[39582.35546875]
135
[36421.69921875]
140
[36606.8359375]
145
[35185.90234375]
150
[38060.3515625]
155
[41041.43359375]
160
[35663.41796875]
165
[31970.748046875]
170
[34289.546875]
175
[35149.34765625]
180
[41133.39453125]
185
[32847.8046875]
190
[34587.859375]
195
[34402.8984375]
200
[32036.71484375]
205
[32645.3359375]
210
[34625.75390625]
215
[35687.984375]
220
[34698.95703125]
225
[35265.05859375]
230
[35319.0390625]
235
[35227.34765625]
240
[34058.015625]
245
[34618.296875]
250
[3